In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

In [3]:
print('review max : {}'.format(max(len(l) for l in X_train)))
print('review avg : {}'.format(sum(map(len, X_train))/len(X_train)))

review max : 2494
review avg : 238.71364


In [4]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [5]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Bahdanau Attention

In [8]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, values, query): # 단, key와 value는 같음
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

BiLSTM with Attention Mechanism

In [9]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, BatchNormalization
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [10]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len)(sequence_input)

순방향 LSTM의 은닉 상태와 셀상태를 forward_h, forward_c에 저장하고, 역방향 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장. 

In [11]:
#BiLSTM
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
    (LSTM
     (128,
      dropout=0.3,
      return_sequences=True,
      return_state=True,
      recurrent_activation='relu',
      recurrent_initializer='glorot_uniform'))(embedded_sequences)

In [12]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 256) (None, 128) (None, 128) (None, 128) (None, 128)


In [13]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [14]:
attention = BahdanauAttention(128) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

hidden = BatchNormalization()(context_vector)

output = Dense(2, activation='softmax')(hidden)
model = Model(inputs=sequence_input, outputs=output)

In [15]:
Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)

model.compile(optimizer=Adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
196/196 [==============================] - 1719s 9s/step - loss: 0.6301 - accuracy: 0.6356 - val_loss: 0.6605 - val_accuracy: 0.7270
Epoch 2/10
196/196 [==============================] - 1702s 9s/step - loss: 0.4651 - accuracy: 0.7810 - val_loss: 0.5169 - val_accuracy: 0.8070
Epoch 3/10
196/196 [==============================] - 1654s 8s/step - loss: 0.3657 - accuracy: 0.8388 - val_loss: 0.3801 - val_accuracy: 0.8479
Epoch 4/10
196/196 [==============================] - 2232s 11s/step - loss: nan - accuracy: 0.5241 - val_loss: nan - val_accuracy: 0.5000
Epoch 5/10
196/196 [==============================] - 1686s 9s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 6/10
196/196 [==============================] - 1645s 8s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 7/10
196/196 [==============================] - 1634s 8s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000
Epoch 8/10
196/

In [17]:
print("\n test accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 181s 231ms/step - loss: nan - accuracy: 0.5000

 test accuracy: 0.5000
